In [1]:
import pandas as pd
import numpy as np
#for text pre-processing
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
#nltk.download('wordnet')
#for model-building
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
# bag of words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
#for word embedding
#import gensim
#from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [2]:
from sklearn.datasets import fetch_20newsgroups
import pandas as pd

def twenty_newsgroup_to_csv():
    newsgroups_train = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))

    df = pd.DataFrame([newsgroups_train.data, newsgroups_train.target.tolist()]).T
    df.columns = ['text', 'target']

    targets = pd.DataFrame( newsgroups_train.target_names)
    targets.columns=['title']

    out = pd.merge(df, targets, left_on='target', right_index=True)
    out['date'] = pd.to_datetime('now')
    out.to_csv('20_newsgroup.csv')

twenty_newsgroup_to_csv()

C:\Users\carlo\.conda\envs\PBE_22\lib\site-packages\pandas\core\arrays\datetimes.py:2199: FutureWarning: The parsing of 'now' in pd.to_datetime without `utc=True` is deprecated. In a future version, this will match Timestamp('now') and Timestamp.now()
  result, tz_parsed = tslib.array_to_datetime(


In [3]:
data = pd.read_csv('20_newsgroup.csv', index_col = 0)
# Select only 5 classes
class_mask = data["target"].isin([7,9,13,18,19])
data = data[class_mask]
data.head()

,text,target,title,date
9,\nIf a Christian means someone who believes in...,19,talk.religion.misc,2022-04-07 15:08:27.454395
10,the blood of the lamb.\n\nThis will be a hard ...,19,talk.religion.misc,2022-04-07 15:08:27.454395
12,930418\n\nDo what thou wilt shall be the whole...,19,talk.religion.misc,2022-04-07 15:08:27.454395
40,\n\nNo. Zeno's paradox is resolved by showing...,19,talk.religion.misc,2022-04-07 15:08:27.454395
58,"\nDefinitely, J.R. ""Bob"" Dobbs, numero uno, to...",19,talk.religion.misc,2022-04-07 15:08:27.454395


In [4]:
#SPLITTING THE TRAINING DATASET INTO TRAIN AND TEST
df_train, df_test = train_test_split(data,test_size=0.1,shuffle=True, random_state=1)

In [5]:
#convert to lowercase, strip and remove punctuations
def preprocess(text):
    text = text.lower()
    text=text.strip()
    text=re.compile('<.*?>').sub('', text)
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)
    text = re.sub('\s+', ' ', text)
    text = re.sub(r'\[[0-9]*\]',' ',text)
    text=re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text)
    text = re.sub(r'\s+',' ',text)
    return text


# STOPWORD REMOVAL
def stopword(string):
    a= [i for i in string.split() if i not in stopwords.words('english')]
    return ' '.join(a)
#LEMMATIZATION
# Initialize the lemmatizer
wl = WordNetLemmatizer()

# This is a helper function to map NTLK position tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
# Tokenize the sentence
def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(word_tokenize(string)) # Get position tags
    a=[wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)] # Map the position tag and lemmatize the word/token
    return " ".join(a)

In [7]:
df_train = df_train.dropna()
df_test = df_test.dropna()

In [8]:
def finalpreprocess(string):
    return lemmatizer(stopword(preprocess(string)))
df_train['clean_text'] = df_train['text'].apply(lambda x: finalpreprocess(x))
df_train.head()
df_test['clean_text'] = df_test['text'].apply(lambda x: finalpreprocess(x))

In [9]:
#SPLITTING THE TRAINING DATASET INTO TRAIN AND TEST
X_train, X_val, y_train, y_val = train_test_split(df_train["clean_text"],df_train["title"],test_size=0.2,shuffle=True, stratify=df_train["title"], random_state=2)
X_test = df_test['clean_text']
y_test = df_test['title']
# Word2Vec runs on tokenized sentences
X_train_tok= [nltk.word_tokenize(i) for i in X_train]
X_val_tok= [nltk.word_tokenize(i) for i in X_val]
X_test_tok= [nltk.word_tokenize(i) for i in X_test]

In [10]:
#Tf-Idf vectorizer
tfidf_vectorizer = TfidfVectorizer(use_idf=True)
X_train_vectors_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_val_vectors_tfidf = tfidf_vectorizer.transform(X_val)
X_test_vectors_tfidf = tfidf_vectorizer.transform(X_test)

In [12]:
nn_tfidf = MLPClassifier()
nn_tfidf.fit(X_train_vectors_tfidf, y_train)
#Predict y value for test dataset
y_predict = nn_tfidf.predict(X_test_vectors_tfidf)
#y_prob = dt_tfidf.predict_proba(X_test_vectors_tfidf)[:,1]
print(classification_report(y_test,y_predict))
print('Confusion Matrix:\n',confusion_matrix(y_test, y_predict))

nn_acc = accuracy_score(y_test, y_predict)
nn_f1 = f1_score(y_test, y_predict, average ='weighted')

print(f"Test Accuracy of Neural Network(tf-idf) is {nn_acc} \n")
print(f"Test F1-score weighted of Neural Network(tf-idf) is {nn_f1} \n")

                    precision    recall  f1-score   support

         rec.autos       0.91      0.96      0.93       100
rec.sport.baseball       0.90      0.93      0.92        88
           sci.med       0.95      0.94      0.94        95
talk.politics.misc       0.87      0.88      0.88        86
talk.religion.misc       0.84      0.70      0.77        54

          accuracy                           0.90       423
         macro avg       0.89      0.88      0.89       423
      weighted avg       0.90      0.90      0.90       423

Confusion Matrix:
 [[96  1  0  2  1]
 [ 2 82  1  2  1]
 [ 3  1 89  1  1]
 [ 1  5  0 76  4]
 [ 4  2  4  6 38]]


NameError: name 'balanced_accuracy_score' is not defined

In [38]:
X_test_vectors_tfidf

<1x23664 sparse matrix of type '<class 'numpy.float64'>'
	with 26 stored elements in Compressed Sparse Row format>

In [13]:
import random

In [78]:
interests = ["autos", "baseball", "medicine", "politics", "religion"]

def generate_users(n_user, interests= ["autos", "baseball", "medicine", "politics", "religion"]):
    users = {}
    for user in range(n_user):
        n_interest = random.randint(1, 3)
        user_interest = []
        i = random.sample(range(0,4), n_interest)
        user_interest.append([interests[e] for e in i])
            
        users[user] = user_interest
    return users
        
    

In [80]:
users = generate_users(10)
print(users)

{0: [['politics']], 1: [['medicine']], 2: [['baseball']], 3: [['baseball', 'medicine', 'politics']], 4: [['medicine']], 5: [['politics', 'baseball', 'autos']], 6: [['medicine', 'autos']], 7: [['autos', 'politics', 'baseball']], 8: [['baseball', 'politics']], 9: [['autos', 'politics']]}


In [82]:
pred = nn_tfidf.predict(X_test_vectors_tfidf[1])
print(pred[0].split('.')[-1])


med


In [83]:
def assigne_text(users, pred):
    if 'med' in pred:
        text = 'medicine'
    elif 'autos' in pred:
        text = 'autos'
    elif 'politics' in pred:
        text = 'politics'
    elif 'religion' in pred:
        text = 'religion'
    elif 'baseball':
        text = 'baseball'
    assigned = []
    for user, interest in users.items():
        if text in interest[0]:
            assigned.append(user)
            
    return assigned, text.upper()
        

In [84]:
assigned, text_topic = assigne_text(users, pred[0])
for user in assigned:
    print("This text is assigned to user {}. It is a {} text.".format(user, assigned_text[1]))

This text is assigned to user 1. It is a medicine text.
This text is assigned to user 3. It is a medicine text.
This text is assigned to user 4. It is a medicine text.
This text is assigned to user 6. It is a medicine text.
